In [3]:
from datasets import load_dataset

raw_dataset = load_dataset('super_glue', 'cb', cache_dir='../datasets/.cache/huggingface_datasets')

Generating train split:   0%|          | 0/250 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/56 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/250 [00:00<?, ? examples/s]

Dataset super_glue downloaded and prepared to ../datasets/.cache/huggingface_datasets/super_glue/cb/1.0.2/d040c658e2ddef6934fdd97deb45c777b6ff50c524781ea434e7219b56a428a7. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [16]:
from openprompt.data_utils import InputExample

dataset = {}
for split in ['train', 'validation', 'test']:
    dataset[split] = []
    for data in raw_dataset[split]:
        input_example = InputExample(
            text_a=data['premise'],
            text_b=data['hypothesis'],
            label=int(data['label']),
            guid=data['idx']
        )
        dataset[split].append(input_example)


In [22]:
print(len(dataset['train']))
print(type(dataset['train']))
print(type(dataset['train'][0]))
print(dataset['train'][0])

250
<class 'list'>
<class 'openprompt.data_utils.utils.InputExample'>
{
  "guid": 0,
  "label": 0,
  "meta": {},
  "text_a": "It was a complex language. Not written down but handed down. One might say it was peeled down.",
  "text_b": "the language was peeled down",
  "tgt_text": null
}



In [23]:
from openprompt.plms import load_plm
plm, tokenizer, model_config, WrapperClass = load_plm("t5", "t5-base")

Downloading:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/850M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

/opt/homebrew/Caskroom/miniforge/base/envs/ai-resesarch/lib/python3.8/site-packages/transformers/models/t5/tokenization_t5.py:163: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [24]:
plm

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseReluDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dr

In [25]:
tokenizer

PreTrainedTokenizer(name_or_path='t5-base', vocab_size=32100, model_max_len=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'additional_special_tokens': ['<extra_id_0>', '<extra_id_1>', '<extra_id_2>', '<extra_id_3>', '<extra_id_4>', '<extra_id_5>', '<extra_id_6>', '<extra_id_7>', '<extra_id_8>', '<extra_id_9>', '<extra_id_10>', '<extra_id_11>', '<extra_id_12>', '<extra_id_13>', '<extra_id_14>', '<extra_id_15>', '<extra_id_16>', '<extra_id_17>', '<extra_id_18>', '<extra_id_19>', '<extra_id_20>', '<extra_id_21>', '<extra_id_22>', '<extra_id_23>', '<extra_id_24>', '<extra_id_25>', '<extra_id_26>', '<extra_id_27>', '<extra_id_28>', '<extra_id_29>', '<extra_id_30>', '<extra_id_31>', '<extra_id_32>', '<extra_id_33>', '<extra_id_34>', '<extra_id_35>', '<extra_id_36>', '<extra_id_37>', '<extra_id_38>', '<extra_id_39>', '<extra_id_40>', '<extra_id_41>', '<extra_id_42>', '<extra_id_43>', '<extra_

In [26]:
model_config

T5Config {
  "_name_or_path": "t5-base",
  "architectures": [
    "T5WithLMHeadModel"
  ],
  "d_ff": 3072,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
      "no_repeat_ngram_size": 3,
      "num_beams": 4,
      "prefix": "summarize: "
    },
    "translation_en_to_de": {
      "early_stopping": true,
      "max_length": 300,
      "num_beams": 4,
      "prefix": "translate English to German: "
    },
    "translatio

In [27]:
WrapperClass

openprompt.plms.seq2seq.T5TokenizerWrapper

In [30]:
from openprompt.prompts import ManualTemplate

template_text = '{"placeholder":"text_a"} Question: {"placeholder":"text_b"}? Is it correct? {"mask"}.'
template = ManualTemplate(tokenizer, template_text)

In [31]:
wrapped_example = template.wrap_one_example(dataset['train'][0])

In [59]:
dataset['train'][0]

{
  "guid": 0,
  "label": 0,
  "meta": {},
  "text_a": "It was a complex language. Not written down but handed down. One might say it was peeled down.",
  "text_b": "the language was peeled down",
  "tgt_text": null
}

In [33]:
from pprint import pprint
# loss ids : (1 -> mask, 0 -> no mask)
# short enable : (0 -> trunc 불가능 (special token or template), 1 -> trunc 가능 (input text))
pprint(wrapped_example)

[[{'loss_ids': 0,
   'shortenable_ids': 1,
   'text': 'It was a complex language. Not written down but handed down. One '
           'might say it was peeled down.'},
  {'loss_ids': 0, 'shortenable_ids': 0, 'text': ' Question:'},
  {'loss_ids': 0,
   'shortenable_ids': 1,
   'text': ' the language was peeled down'},
  {'loss_ids': 0, 'shortenable_ids': 0, 'text': '? Is it correct?'},
  {'loss_ids': 1, 'shortenable_ids': 0, 'text': '<mask>'},
  {'loss_ids': 0, 'shortenable_ids': 0, 'text': '.'}],
 {'guid': 0, 'label': 0}]


In [34]:
dataset['train'][0]

{
  "guid": 0,
  "label": 0,
  "meta": {},
  "text_a": "It was a complex language. Not written down but handed down. One might say it was peeled down.",
  "text_b": "the language was peeled down",
  "tgt_text": null
}

In [35]:
wrapped_t5_tokenizer = WrapperClass(max_seq_length=128, decoder_max_length=3, tokenizer=tokenizer, truncate_method="head")

In [40]:
tokenized_example = wrapped_t5_tokenizer.tokenize_one_example(wrapped_example, teacher_forcing=False)

In [50]:
print(tokenized_example.keys())
print(type(tokenized_example['input_ids']))
print(len(tokenized_example['input_ids']))
print(type(tokenized_example['attention_mask']))
print(len(tokenized_example['attention_mask']))
print(type(tokenized_example['decoder_input_ids']))
print(len(tokenized_example['decoder_input_ids']))
print(type(tokenized_example['loss_ids']))
print(len(tokenized_example['loss_ids']))

dict_keys(['input_ids', 'attention_mask', 'decoder_input_ids', 'loss_ids'])
<class 'list'>
128
<class 'list'>
128
<class 'list'>
3
<class 'list'>
3


In [52]:
print(tokenizer.convert_ids_to_tokens(tokenized_example['input_ids']))
print(tokenizer.convert_ids_to_tokens(tokenized_example['decoder_input_ids']))

['▁It', '▁was', '▁', 'a', '▁complex', '▁language', '.', '▁Not', '▁written', '▁down', '▁but', '▁handed', '▁down', '.', '▁One', '▁might', '▁say', '▁it', '▁was', '▁pe', 'ele', 'd', '▁down', '.', '▁Question', ':', '▁the', '▁language', '▁was', '▁pe', 'ele', 'd', '▁down', '▁', '?', '▁I', 's', '▁it', '▁correct', '?', '<extra_id_0>', '▁', '.', '</s>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>

In [88]:
# from collections import Counter

model_inputs = {}
for split in ['train', 'validation', 'test']:
    model_inputs[split] = []
    for sample in dataset[split]:
        tokenized_example = wrapped_t5_tokenizer.tokenize_one_example(template.wrap_one_example(sample), teacher_forcing=False)
        model_inputs[split].append(tokenized_example)
        # counter = Counter(tokenized_example['attention_mask'])
        # if counter[0] == 0:
        #     print(len(sample.text_a))
            # print(len(sample.text_b))

In [89]:
from openprompt import PromptDataLoader

train_data_loader = PromptDataLoader(
    dataset=dataset['train'],
    template=template,
    tokenizer=tokenizer,
    tokenizer_wrapper_class=WrapperClass,
    max_seq_length=256,
    decoder_max_length=3,
    batch_size=4,
    shuffle=True,
    teacher_forcing=False,
    predict_eos_token=False,
    truncate_method="head"
)

tokenizing: 250it [00:00, 1598.11it/s]


In [91]:
from openprompt.prompts import ManualVerbalizer
import torch

my_verbalizer = ManualVerbalizer(tokenizer, num_classes=3, label_words=[["yes"], ["no"], ["maybe"]])

In [95]:
my_verbalizer.label_words

[[' yes'], [' no'], [' maybe']]

In [97]:
logits = torch.randn(2, len(tokenizer))

In [99]:
logits.shape

torch.Size([2, 32100])

In [103]:
print(my_verbalizer.process_logits(logits))
print(my_verbalizer.process_logits(logits).shape)

tensor([[-1.2690, -2.8744, -0.4118],
        [-1.6407, -1.5485, -0.5216]])
torch.Size([2, 3])


In [106]:
from openprompt import PromptForClassification

use_cuda = torch.cuda.is_available()
prompt_model = PromptForClassification(plm=plm, template=template, verbalizer=my_verbalizer, freeze_plm=False)

In [108]:
if use_cuda:
    prompt_model = prompt_model.cuda()

In [114]:
from transformers import AdamW, get_linear_schedule_with_warmup

loss = torch.nn.CrossEntropyLoss()
no_decay = ['bias', 'LayerNorm.weight']

In [126]:
optimizer_grouped_parameters = [
    {'params': [p for n, p in prompt_model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in prompt_model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [141]:
optimizer = AdamW(optimizer_grouped_parameters, lr=1e-4)

for epoch in range(10):
    total_loss = 0
    for step, inputs in enumerate(train_data_loader):
        if use_cuda:
            inputs = inputs.cuda()

        logits = prompt_model(inputs)
        labels = inputs['label']
        loss = loss(logits, labels)
        loss.backward()
        total_loss += loss.item()
        optimizer.step()
        optimizer.zero_grad()

        if step % 100 == 1:
            print(f"Epoch {epoch}, average loss: {total_loss / (step + 1)}")


/opt/homebrew/Caskroom/miniforge/base/envs/ai-resesarch/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


KeyboardInterrupt: 